# Parametrized Fully Connected Network in Fugu

This notebook builds a fully connected network (input → hidden → output) using Fugu bricks and scaffold, runs it on the snn_Backend for 20 steps, and verifies connectivity.

In [10]:
import numpy as np
from fugu import Scaffold
from fugu.bricks.input_bricks import Vector_Input
from fugu.bricks.dense_bricks import dense_layer_1d
from fugu.backends import snn_Backend

# Parametrize network dimensions

input_size = 2   
output_size = 2
n_steps = 20

# Create dummy input spikes 
spikes = np.zeros((input_size, n_steps))
for i in range(input_size):
    spikes[i, i % n_steps] = 1

# Build scaffold and add input brick 
scaffold = Scaffold()
scaffold.add_brick(
    Vector_Input(spikes, coding='Raster', name='Input', time_dimension=True),
    input_nodes='input'
 )

# Add output layer brick (fully connected to input) 
W = np.ones((output_size, input_size)) * 0.7  # shape (2, 4)
T = np.ones(output_size) * 0.9                # shape (2,)
scaffold.add_brick(
    dense_layer_1d(output_shape=(output_size,), weights=W, thresholds=T, name='hidden0'),
    input_nodes=[-1],
    output=False
)
scaffold.add_brick(
    dense_layer_1d(output_shape=(output_size,), weights=W, thresholds=T, name='hidden1'),
    input_nodes=[-1],
    output=False
)
scaffold.add_brick(
    dense_layer_1d(output_shape=(output_size,), weights=W, thresholds=T, name='Output'),
    input_nodes=[-1],
    output=True
)

print("Weights shape:", W.shape)
print("Thresholds shape:", T.shape)

 #Build the network graph 
scaffold.lay_bricks()

# print all edges 
print('Network edges (source → target):')
for u, v, d in scaffold.graph.edges(data=True):
    print(f'{u} → {v} | weight={d.get("weight")} | delay={d.get("delay")}')

#  Run simulation 
backend = snn_Backend()
# backend = loihi_backend()
backend.compile(scaffold, compile_args={'record': 'all', 'debug_mode': False})
result = backend.run(n_steps=n_steps)

# Show result summary 
print('Simulation result (spike times):')
print(result)

Weights shape: (2, 2)
Thresholds shape: (2,)
Network edges (source → target):
Input-20:begin → Input-20:complete | weight=1.0 | delay=19
Input-20:begin → hidden0_begin | weight=0.0 | delay=1
Input-20:complete → hidden0_complete | weight=0.0 | delay=1
Input-20:(0,) → hidden0d0 | weight=0.7 | delay=1
Input-20:(0,) → hidden0d1 | weight=0.7 | delay=1
Input-20:(1,) → hidden0d0 | weight=0.7 | delay=1
Input-20:(1,) → hidden0d1 | weight=0.7 | delay=1
hidden0_begin → hidden1_begin | weight=0.0 | delay=1
hidden0_complete → hidden1_complete | weight=0.0 | delay=1
hidden0d0 → hidden1d0 | weight=0.7 | delay=1
hidden0d0 → hidden1d1 | weight=0.7 | delay=1
hidden0d1 → hidden1d0 | weight=0.7 | delay=1
hidden0d1 → hidden1d1 | weight=0.7 | delay=1
hidden1_begin → Output_begin | weight=0.0 | delay=1
hidden1_complete → Output_complete | weight=0.0 | delay=1
hidden1d0 → Outputd0 | weight=0.7 | delay=1
hidden1d0 → Outputd1 | weight=0.7 | delay=1
hidden1d1 → Outputd0 | weight=0.7 | delay=1
hidden1d1 → Outputd

In [ ]:
import networkx as nx
print(nx.__version__)

3.5
